In [ ]:
import PyPDF2 as pdf
import os


def main():
    dirName = input(
        "Enter the directory name which papers with txt file extension are located: ")
    files = load_files(dirName)


In [ ]:
def convert_pdfs_to_txt(directory):
    """
    Extract texts from pdf files and write their content into new txt files
    You only want to do this process once
    Inputs a directory name, for every pdf file in the directory
    this function creates new .txt files writes the content of pdf
    """
    for file in os.listdir(directory):
        pdfFile = open(os.path.join(directory, file), "rb")
        pdfReader = pdf.PdfFileReader(pdfFile)
        numPdfPage = pdfReader.numPages
        paperTxt = []
        filetxtName = file[:-3]
        for pageNum in range(numPdfPage):
            paperTxt.append(pdfReader.getPage(pageNum).extractText())
        with open(os.path.join("papersTXT", filetxtName), "w", encoding="utf-8") as f:
            for pagetext in paperTxt:
                f.write(pagetext)
    return True


convert_pdfs_to_txt("papersPDF")


In [ ]:
def load_files(directory):
    """
    Extract text from txt files
    Inputs a directory name, 
    Outputs a dictionary whose keys are names of the files and values are texts of the files
    """
    filesDict = dict()
    for file in os.listdir(directory):
        with open(os.path.join(directory, file), "r", encoding="utf-8") as f:
            filesDict[file] = f.read()
    return filesDict


In [ ]:
if __name__ == "__main__":
    main()
